**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 2.8MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N207e65bdf4b542d18bc65565ddac3c1d (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [44]:
#For this part since I am asked to list ALL subclasses, I will try to show even the subclasses of subclasses of the Person Class.
base = Namespace("http://somewhere#")
print("Subclasses list using RDFLib")
for s,p,o in g.triples((None, RDFS.subClassOf, base.Person)):
  print(s,"is a subclass of Person")
for s,p,o in g.triples((None, RDFS.subClassOf, base.Person)):
  for s0,p0,o0 in g.triples((None, RDFS.subClassOf, s)):
    print(s0,"is a subclass of a subclass of Person")
#I'll use the rdfs:subClass* to ask for all the possible subclasses of Person
qres = g.query(
"""SELECT DISTINCT ?subClass ?p ?o
WHERE {
?subClass rdfs:subClassOf* <http://somewhere#Person>.
?subClass ?p ?o filter(?p not in ( rdf:type ))
}""")
print("Subclasses list using SPARQL")
for row in qres:
  print("%s is a subclass of Person" % row)


Subclasses list using RDFLib
http://somewhere#Researcher is a subclass of Person
http://somewhere#PhDstudent is a subclass of a subclass of Person
Subclasses list using SPARQL
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person is a subclass of Person
http://somewhere#PhDstudent http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Researcher is a subclass of Person


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [35]:
print("Individuals list using RDFLib")
for s,p,o in g.triples((None, RDFS.subClassOf, base.Person)):
  for s0,p0,o0 in g.triples((None,RDF.type,s)):
    print(s0,"is an individual of Person")
for s,p,o in g.triples((None, RDF.type,base.Person)):
  print(s,"is an individual of Person")
print("Individuals list using SPARQL")
qres = g.query(
"""SELECT DISTINCT ?individual
WHERE {
  {?person rdfs:subClassOf* <http://somewhere#Person> .
  ?individual a ?person. } UNION
  {?individual  a <http://somewhere#Person>.}
}""")
for row in qres:
  print("%s is an individual of Person" % row)

Individuals list using RDFLib
http://somewhere#JaneSmith is an individual of Person
http://somewhere#SaraJones is an individual of Person
http://somewhere#JohnSmith is an individual of Person
Individuals list using SPARQL
http://somewhere#SaraJones is an individual of Person
http://somewhere#JohnSmith is an individual of Person
http://somewhere#JaneSmith is an individual of Person


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [34]:
print("Individuals and properties list using RDFLib")
for s,p,o in g.triples((None, RDFS.subClassOf, base.Person)):
  for s0,p0,o0 in g.triples((None,RDF.type,s)):
    for s1,p1,o1 in g.triples((s0,None,None)):
      print(s1,p1,o1)
for s,p,o in g.triples((None,RDF.type,base.Person)):
  for s0,p0,o0 in g.triples((s,None,None)):
    print(s0,p0,o0)
print("Individuals and properties list using SPARQL")
qres = g.query(
"""SELECT DISTINCT ?individual ?prop ?value
WHERE {
  {?person rdfs:subClassOf* <http://somewhere#Person>.
   ?individual a ?person.
   ?individual ?prop ?value.
   } UNION
  {?individual  a <http://somewhere#Person>.
  ?individual ?prop ?value.}
}""")
for row in qres:
  print("%s %s %s" % row)

Individuals and properties list using RDFLib
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http